## Data enriching tasks

We add some data about global peace index and global terrorism index through the year in our dataset to check well the impacted of these index of user's session evolution.

In [5]:
from warnings import filterwarnings

filterwarnings("ignore")

In [6]:
import os

os.chdir("/workspaces/Humanitarian-activity-analysis/")

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as geopd
import numpy as np

In [8]:
activity_by_month = pd.read_csv("Data processed/final_project_activity_by_month.csv",
                               parse_dates=["time"],
                               infer_datetime_format=True)

gpi_score = pd.read_csv("Data processed/gpi_score_cleaned.csv")

gti_score = pd.read_csv("Data processed/gti_score_cleaned.csv")

In [9]:
activity_by_month.head(2)

,_geojson,project_id,status,organisation,Country,time,sessions,sessions_mapped,sessions_validated,cum_sessions,cum_sessions_mapped,cum_sessions_validated
0,"{""type"":""Feature"",""properties"":{""project_id"":9...",96,0,NaN,Indonesia,2012-11-01,118,86,32,118,86,32
1,"{""type"":""Feature"",""properties"":{""project_id"":9...",99,0,NaN,Uganda,2012-11-01,12,11,0,12,11,0


In [10]:
gpi_score.head(2)

,Country,gpi_iso3c,gpi_2008,gpi_2009,gpi_2010,gpi_2011,gpi_2012,gpi_2013,gpi_2014,gpi_2015,gpi_2016,gpi_2017,gpi_2018,gpi_2019,gpi_2020,gpi_2021,gpi_2022
0,Afghanistan,AFG,3.095,3.191,3.04,3.021,3.154,3.258,3.212,3.316,3.362,3.412,3.513,3.520,3.592,3.600,3.554
1,Albania,ALB,1.806,1.779,1.77,1.797,1.799,1.816,1.810,1.806,1.761,1.774,1.714,1.712,1.771,1.769,1.761


In [11]:
activity_by_month["Year"] = activity_by_month["time"].dt.year

activity_by_month.head(2)

,_geojson,project_id,status,organisation,Country,time,sessions,sessions_mapped,sessions_validated,cum_sessions,cum_sessions_mapped,cum_sessions_validated,Year
0,"{""type"":""Feature"",""properties"":{""project_id"":9...",96,0,NaN,Indonesia,2012-11-01,118,86,32,118,86,32,2012
1,"{""type"":""Feature"",""properties"":{""project_id"":9...",99,0,NaN,Uganda,2012-11-01,12,11,0,12,11,0,2012


In [12]:
activity_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63994 entries, 0 to 63993
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _geojson                63994 non-null  object        
 1   project_id              63994 non-null  int64         
 2   status                  63994 non-null  int64         
 3   organisation            46149 non-null  object        
 4   Country                 63994 non-null  object        
 5   time                    63994 non-null  datetime64[ns]
 6   sessions                63994 non-null  int64         
 7   sessions_mapped         63994 non-null  int64         
 8   sessions_validated      63994 non-null  int64         
 9   cum_sessions            63994 non-null  int64         
 10  cum_sessions_mapped     63994 non-null  int64         
 11  cum_sessions_validated  63994 non-null  int64         
 12  Year                    63994 non-null  int64 

In [13]:
gpi_score.head(2)

,Country,gpi_iso3c,gpi_2008,gpi_2009,gpi_2010,gpi_2011,gpi_2012,gpi_2013,gpi_2014,gpi_2015,gpi_2016,gpi_2017,gpi_2018,gpi_2019,gpi_2020,gpi_2021,gpi_2022
0,Afghanistan,AFG,3.095,3.191,3.04,3.021,3.154,3.258,3.212,3.316,3.362,3.412,3.513,3.520,3.592,3.600,3.554
1,Albania,ALB,1.806,1.779,1.77,1.797,1.799,1.816,1.810,1.806,1.761,1.774,1.714,1.712,1.771,1.769,1.761


In [14]:
def get_problematic_country(df_aggs):
    """
    Take a dataframe and look for countries that contains space in their name

    """
    i=0

    dict_other = {}

    for df in df_aggs:
        i+=1
        dict_other[i] = set()
        
        for country in df.Country.unique():

            if " " in country:

                dict_other[i].add(country)
        
    return dict_other


def get_diff(dict_other):

    """
        Take the problematic country's names and restitute the names that different write in the same
        ways or just the names that are not shared in common of both dataframe. 
    """
    
    diff12 = dict_other[1].difference(dict_other[2])
    diff21 = dict_other[2].difference(dict_other[1])
    
    return diff12, diff21

In [15]:
activity_by_month[activity_by_month.Country.str.startswith("Ze")]

,_geojson,project_id,status,organisation,Country,time,sessions,sessions_mapped,sessions_validated,cum_sessions,cum_sessions_mapped,cum_sessions_validated,Year


In [16]:
# Normalisation of country's name that exist in both datasets

activity_by_month["Country"] = activity_by_month["Country"].replace('Bosnia and Herzegovina','Bosnia & Herzegovina')
activity_by_month["Country"] = activity_by_month["Country"].replace('Trinidad and Tobago','Trinidad & Tobago')
activity_by_month["Country"] = activity_by_month["Country"].replace('The Gambia','Gambia')
activity_by_month["Country"] = activity_by_month["Country"].replace('Iran (Islamic Republic of)','Iran')
activity_by_month["Country"] = activity_by_month["Country"].replace('Congo-Brazzaville','Republic of the Congo')
activity_by_month["Country"] = activity_by_month["Country"].replace("Côte d'Ivoire",'Côte d’Ivoire')

gpi_score["Country"] = gpi_score["Country"].replace('Côte d??voire','Côte d’Ivoire')

In [17]:
dict_other = get_problematic_country([activity_by_month, gti_score])

get_diff(dict_other)

({'American Samoa',
  'Antigua and Barbuda',
  'British Virgin Islands',
  'Cayman Islands',
  'East Timor',
  'French Guiana',
  'Marshall Islands',
  'Netherlands Antilles',
  'Northern Mariana Islands',
  'Puerto Rico',
  'Saint Lucia',
  'Saint Vincent and the Grenadines',
  'Solomon Islands',
  'The Bahamas',
  'United States Virgin Islands'},
 {'Czech Republic',
  'New Zealand',
  'North Korea',
  'Saudi Arabia',
  'South Korea',
  'United Arab Emirates'})

In [18]:
temp_df2 = gpi_score[gpi_score["Country"] == "Albania"]

temp_df2["gpi_2008"]

1    1.806
Name: gpi_2008, dtype: float64

In [23]:
def fill_with_external_data(df1, df2, suffix):
    
    to_add_col = suffix+"_index"
    
    dict_to = {"Country":[],"Year":[],suffix:[]}
    
    for country in df1.Country.unique():
        
        temp_df2 = df2[df2["Country"] == country]
        
        for year in df1.Year.unique():

            temp_col = suffix+"_"+str(year)

            if temp_col in temp_df2.columns:
                
                if len(temp_df2[temp_col].values) == 1:

                    dict_to["Country"].append(country)
                    dict_to["Year"].append(year)
                    dict_to[suffix].append(temp_df2[temp_col].values.item())
            
            else:
                
                dict_to["Country"].append(country)
                dict_to["Year"].append(year)
                dict_to[suffix].append(np.NaN)
    
    outputfile = pd.DataFrame(dict_to)
    
    temp = pd.merge(df1, outputfile)
            
    return temp

In [26]:
temp_outputfile = fill_with_external_data(activity_by_month,gpi_score,suffix="gpi")

outputfile = fill_with_external_data(temp_outputfile,gti_score,suffix="gti")

In [27]:
outputfile.head(3)

,_geojson,project_id,status,organisation,Country,time,sessions,sessions_mapped,sessions_validated,cum_sessions,cum_sessions_mapped,cum_sessions_validated,Year,gpi,gti
0,"{""type"":""Feature"",""properties"":{""project_id"":9...",96,0,NaN,Indonesia,2012-11-01,118,86,32,118,86,32,2012,1.685,5.307074
1,"{""type"":""Feature"",""properties"":{""project_id"":9...",99,0,NaN,Uganda,2012-11-01,12,11,0,12,11,0,2012,2.277,4.641149
2,"{""type"":""Feature"",""properties"":{""project_id"":9...",99,0,NaN,Uganda,2012-12-01,77,53,12,89,64,12,2012,2.277,4.641149


In [28]:
outputfile.isna().sum()

_geojson                      0
project_id                    0
status                        0
organisation              17131
Country                       0
time                          0
sessions                      0
sessions_mapped               0
sessions_validated            0
cum_sessions                  0
cum_sessions_mapped           0
cum_sessions_validated        0
Year                          0
gpi                         155
gti                        9545
dtype: int64

In [29]:
outputfile = outputfile.dropna()

outputfile.head()

,_geojson,project_id,status,organisation,Country,time,sessions,sessions_mapped,sessions_validated,cum_sessions,cum_sessions_mapped,cum_sessions_validated,Year,gpi,gti
9,"{""type"":""Feature"",""properties"":{""project_id"":1...",113,0,OSM RDC,Democratic Republic of the Congo,2012-12-01,18,12,5,18,12,5,2012,2.934,5.825723
35,"{""type"":""Feature"",""properties"":{""project_id"":3...",307,0,Other,Kenya,2013-09-01,56,49,7,56,49,7,2013,2.520,6.974787
36,"{""type"":""Feature"",""properties"":{""project_id"":3...",307,0,Other,Kenya,2013-10-01,15,15,0,71,64,7,2013,2.520,6.974787
37,"{""type"":""Feature"",""properties"":{""project_id"":3...",307,0,Other,Kenya,2013-11-01,25,25,0,96,89,7,2013,2.520,6.974787
57,"{""type"":""Feature"",""properties"":{""project_id"":1...",105,0,HOT,Indonesia,2013-01-01,22,22,0,22,22,0,2013,1.690,5.167678


In [405]:
outputfile.shape

(35619, 15)

In [406]:
outputfile.to_csv("Data processed/Cleaned_Enriched_activity_data.csv", index = False)